In [ ]:
# OpenAI GYM
# Stable-baselines 3 
# Optuna
# AWS Sagemaker Stuido 


In [35]:
# Install system packages
!apt-get update && apt-get -y install unzip freeglut3-dev cmake libopenmpi-dev python3-dev zlib1g-dev swig python3-dev libopenmpi-dev xvfb x11-utils ffmpeg #remove -qq for full out

# Install pyton packages : OBS Box2D might require kernel restart
!pip3 install stable-baselines3[extra] box2d box2d-py box2d-kengz pyvirtualdisplay pyglet==1.5.0 numpy optuna 

Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu xenial InRelease                        
Get:9 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]       
Get:10 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Fetched 325 kB in 1s (272 kB/s)   
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.5.1-3).
unzip is already the newest version (6.0-20ubuntu1).
x11-utils is already the newest ve

In [1]:
import gym
import numpy as np
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
import os
import optuna


# Video 
from pathlib import Path
from IPython import display as ipythondisplay

# Stable baselines

# TD3
from stable_baselines3 import TD3
from stable_baselines3.td3.policies import MlpPolicy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# PPO 
# from stable_baselines3 import PPO
# from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CallbackList, BaseCallback, CheckpointCallback, EveryNTimesteps, EvalCallback, StopTrainingOnRewardThreshold, StopTrainingOnMaxEpisodes

In [2]:
# Values
env_id = 'LunarLanderContinuous-v2'
video_folder = './videos'
video_length = 3000
logs_base_dir = "./log" # Log DIR
steps_total= 0 # Keep track of total steps
os.makedirs(logs_base_dir, exist_ok=True) 

# Settings
log_interval=100

score = 0



### Enviorment 

#TD3 

env = gym.make(env_id)
eval_env = gym.make(env_id)
# env = make_vec_env(env_id, n_envs=1)
# eval_env = make_vec_env(env_id, n_envs=1)
# Logs will be saved in log_dir/monitor.csv
# env = Monitor(env, logs_base_dir)

print('State shape: ', env.observation_space.shape)
# print('Number of actions: ', env.action_space.n)


State shape:  (8,)


In [3]:
# # Tuned
# LunarLander-v2:
#   n_timesteps: !!float 1e5
#   policy: 'MlpPolicy'
#   learning_rate: !!float 6.3e-4
#   batch_size: 128
#   buffer_size: 50000
#   learning_starts: 0
#   gamma: 0.99
#   target_update_interval: 250
#   train_freq: 4
#   gradient_steps: -1
#   exploration_fraction: 0.12
#   exploration_final_eps: 0.1
#   policy_kwargs: "dict(net_arch=[256, 256])"

In [4]:
### Record & Display Video

import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

# Record video
def record_model(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  eval_env = VecVideoRecorder(env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)
  eval_env.close()


## Display video
def display_videos(video_path='', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [5]:
        # Training function
def train_model(name, steps=10000, save=False, prefix=env_id, eval=1000):
    model.learn(total_timesteps=steps, log_interval=log_interval)
    if save == True:
     model.save(name + "-" + prefix)

def record_video(name, length=1500):
    record_model(env_id, model, video_length=length, prefix=name)
    display_videos('videos', prefix=name)
    print(name, " steps total")

In [6]:
def run_training(steps_per_round=0,limit=0):

  global score
  global steps_total
  
  while score < limit:
      steps_total = steps_total + steps_per_round
      train_model(str(steps_total), steps=steps_per_round)
      new_evaluation = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
      score = new_evaluation[0]
      record_video(name=steps_total, length=1000) # uncomment to show video from each round
      print("Mean reward:", score )
    

  # Threshold reached > evaluate over 100 episodes > Video rec/display
  print("Reward limit achived, messuring over 100ep & recording video, please wait...")
  record_video(name=steps_total, length=1750)
  ep100 = evaluate_policy(model, eval_env, n_eval_episodes=50, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
  print("Mean Reward 100 Epispodes: ", ep100[0])
  return steps_total

In [7]:
def run_training_no_video(steps_per_round=200000,limit=300, max_steps=999999999):

  
  global score
  global steps_total
  
  # reset steps_total & score before running score loop
  steps_total = 0
  score = 0
  
  
  while score < limit:
      steps_total = steps_total + steps_per_round
      train_model(str(steps_total), steps=steps_per_round)
      new_eval = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
      score = new_eval[0]
      print("Round completed, steps_total: ",steps_total, " mean: ", score )  
        
      # If score reached, break         
      if score >= limit:
          print("Score limit of ",score," reached")
          break
        
      # If steos limit reached, break
      if steps_total >= max_steps:
          print("Max steps limit of ",max_steps," with a total count of steps_total: ",steps_total,)
          break
    

  # Threshold reached > evaluate over 100 episodes > Video rec/display

  ep100 = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
  print("Mean reward 100 ep: ", ep100[0])
  print("Steps total:", steps_total)
  print(score)
  return score  

In [8]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

os.makedirs(logs_base_dir, exist_ok=True)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(logs_base_dir))

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ea0d122c35ce.ngrok.io


In [ ]:
env.reset()
# !apt-get install python-mysqldb -y
# !conda install mysqldb
# !pip3 install mysql-connector redis
# import mysql.connector
import optuna
import redis


storage = optuna.storages.RedisStorage(
    url="redis://localhost:6379",
)
    
    
    
study = optuna.create_study(study_name="lunarlanderv3",storage=storage, load_if_exists=True)
study.optimize(objective, n_trials=20)
print(study.best_params)
print(study.best_trial)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: ExperimentalWarning: RedisStorage is experimental (supported from v1.4.0). The interface can change in the future.
  # This is added back by InteractiveShellApp.init_path()
[I 2020-11-16 07:14:40,068] Using an existing study with name 'lunarlanderv3' instead of creating a new one.
/opt/conda/lib/python3.6/site-packages/stable_baselines3/common/off_policy_algorithm.py:138: UserWarning: You passed a positive value for `train_freq` and `n_episodes_rollout`.Please make sure this is intended. The agent will collect data by stepping in the environment until both conditions are true: `number of steps in the env` >= `train_freq` and `number of episodes` > `n_episodes_rollout`
  "You passed a positive value for `train_freq` and `n_episodes_rollout`."


Round completed, steps_total:  100000  mean:  -311.04685711795844
Round completed, steps_total:  200000  mean:  -328.39313347988866
Round completed, steps_total:  300000  mean:  -273.860813326389
Round completed, steps_total:  400000  mean:  -327.1873476711945
Round completed, steps_total:  500000  mean:  -313.9768396336156
Round completed, steps_total:  600000  mean:  -288.0673919823515
Round completed, steps_total:  700000  mean:  -299.88463346581216
Round completed, steps_total:  800000  mean:  -266.3157691487225
Round completed, steps_total:  900000  mean:  -374.10870239651626
Round completed, steps_total:  1000000  mean:  -318.3785855861563
Round completed, steps_total:  1100000  mean:  -273.88468735682966
Round completed, steps_total:  1200000  mean:  -281.8355832447261
Round completed, steps_total:  1300000  mean:  -324.6167135683926
Round completed, steps_total:  1400000  mean:  -319.72320273051486
Round completed, steps_total:  1500000  mean:  -335.27685106970625
Round complet

[I 2020-11-16 07:46:50,999] Trial 3 finished with value: 2000000.0 and parameters: {'gamma': 0.999, 'lr': 0.022984280488264328, 'learning_starts': 100000, 'batch_size': 32, 'buffer_size': 1000000, 'train_freq': 2000, 'noise_type': 'normal', 'noise_std': 0.28964279174003305}. Best is trial 3 with value: 2000000.0.


Mean reward 100 ep:  [-310.9632294127716, -352.79760528679844, -380.512596520814, -136.14038012399405, -420.12270917479185, -313.39042821208284, -194.21614111341245, -296.1455503338326, -399.1563477478997, -1419.8328242624195]
Steps total: 2000000
-271.3939005374057
Round completed, steps_total:  100000  mean:  -637.7018852713053
Round completed, steps_total:  400000  mean:  -645.1373685959321
